# Benchmark model generation

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import os


# Views 3
import views_runs
from views_forecasts.extensions import *


In [ ]:
# Common parameters:

dev_id = 'Fatalities002'
run_id = 'Fatalities002'
EndOfHistory = 508
level = 'cm'
get_future = False

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/ViEWS predicting fatalities/Tables/'



print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)

In [ ]:

# Fatalities002 stuff - contains the list of the current fatalities002 ensemble models

from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels(level)
    
i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

In [ ]:
# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

ModelList = RetrieveStoredPredictions(ModelList, steps, EndOfHistory, dev_id, level, get_future)

ModelList = CalibratePredictions(ModelList, EndOfHistory, steps)

In [ ]:
ModelList[0]['predictions_test_df']
ModelList[0]

In [ ]:
# Dataframe with actuals
df_actuals = pd.DataFrame(ModelList[0]['predictions_test_df']['ln_ged_sb_dep'])
print(df_actuals.head())
print(df_actuals.tail())


In [ ]:
def reshape_df(df, draw):
    df = df.drop('ln_ged_sb_dep',axis=1)
    df.reset_index(inplace=True)
    df['draw'] = draw
    df_long = pd.wide_to_long(df, 'step_pred_', i = ['month_id', 'country_id', 'draw'], j = 'step')
    return(df_long)
    
model_draw = 0
df = ModelList[model_draw]['predictions_test_df'].copy()
df_results_long = reshape_df(df,model_draw)
print('Starting with model/draw',model_draw, model['modelname'])
print(df_results_long.describe())


for model in ModelList[1:]:
    model_draw += 1
    print('Appending model/draw',model_draw, model['modelname'])
    df = ModelList[model_draw]['predictions_test_df'].copy()
    df_reshaped = reshape_df(df,model_draw)
    df_results_long = pd.concat([df_results_long ,df_reshaped], axis=0)
    

In [ ]:
# Results file in long format
print(df_results_long.describe())
print(df_results_long.tail())
